In [ ]:
import glob
import MDAnalysis as mda
from matplotlib import pyplot as plt
import pandas as pd
import re
from collections import Counter
from scipy.spatial import distance_matrix
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster


<font color='red'>****Number one location of error - dependency on fpocket*</font>


## Analyze fpocket output

In [ ]:
# Conda activate mdanalysis
# Provide Paths

apo_path=('/home/al2108/rds/hpc-work/dd_proj/alpha_syn/input_pdbs/apo/')
holo_path=('/home/al2108/rds/hpc-work/dd_proj/alpha_syn/input_pdbs/holo/')

In [ ]:
from finding_pockets import *
from pocket_match import *


In [ ]:
list_num =[re.split(r'pair|.pdb',entry)[-2] if len(re.split(r'pair|.pdb', entry)) >= 3 else None for entry in os.listdir(holo_path)]
#count=(len([entry for entry in os.listdir(holo_path) if os.path.isfile(os.path.join(holo_path, entry))]))

path=[]
#for name_apo,name_holo in zip(sorted(glob.glob(apo_path+'traj*/')),(sorted(glob.glob(holo_path+'traj*/')))): # only for abeta traj folder
for name_apo,name_holo in zip(sorted(glob.glob(apo_path)),(sorted(glob.glob(holo_path)))):
    try:
        x=name_apo
        y=name_holo
        for pdb_number in (list_num[:50]):
            apo_pockets=x+'pair'+str(pdb_number)+'_out/pair'+str(pdb_number)+'_out.pdb'
            holo_pdb= y+ 'pair'+ str(pdb_number)+ '.pdb'
            get_apo_centroids = Centroid_fpocket(apo_pockets,'resname STP')
            apo_centroids = get_apo_centroids.calculate_centroid()
            apo_holo=Compare_centroids(apo_centroids,holo_pdb,pdb_number)
            distance_holo_apo=apo_holo.calculate_distance()
            path.append(distance_holo_apo)
    except:
        pass

path=list(filter(lambda x: len(x[0]) > 0, path))

        ## Need to make sure that apo_pockets is pointing to the pair_out.pdb output from fpocket output & holo is pointing to the SAME structure but in holo form

In [ ]:
pocket_df=pd.DataFrame({"pdb_file":([item[1] for item in path])})
pocket_df['STP_pockets'] = pd.Series(([(item[0]) for item in path]))

## Extract coordinates of pocket matches
##### From original true holo strucutre (pdb file)
###### _(this is very redundant)_

In [ ]:
pocket_paths_resid = []

for i, row in pocket_df.iterrows():
    pocket_path = holo_path + "pair" + str(row['pdb_file']) + ".pdb"
    pocket_paths_resid.append((Retrive_pocket_residues(pocket_path).return_path_resids()))#[0][1]) #### Remove [0][1] if you want the path included
    

## Using PocketMatch Software to generate similarity Matrix
### Generating PDB files that will be input for PocketMatch Software


In [ ]:
#### Provide a path to place new pdb files
sample_path=('/home/al2108/rds/hpc-work/dd_proj/abeta_ligand/sample_files/')

flat_pocket_paths_resid = [num for sublist in pocket_paths_resid for num in sublist]
fixed_flat_pocket_paths_resid = [(x[0].replace("holo", "apo"), x[1]) for x in flat_pocket_paths_resid]


for f in fixed_flat_pocket_paths_resid[:4]:
    pdb_writer=Writing_pocket_pdb(f,sample_path)
    pdb_writer.generate_pdb()

In [ ]:
directory_pm=(os.getcwd())+'/PocketMatch'

In [ ]:
%%bash -s "$directory_pm" "$sample_path"

cd $1
bash Step0-cabbage.sh $2
./Step3-PM_typeA outfile.cabbage



### Analyzing Output from Pocket Match

In [ ]:
## Ouput of PocketMatch Score
score=directory_pm+'/PocketMatch_score.txt'
linkage_matrix,unique_ids=Evaluating_PM(score).similarity_mat()

In [ ]:
plt.figure(figsize=(10, 5))
dendrogram(linkage_matrix, labels=unique_ids, orientation='right')
plt.xlabel('Distance')
plt.ylabel('PDB Files')
plt.title('Dendrogram of Hierarchical Clustering')
plt.show()

In [ ]:
cluster_dict=Evaluating_PM(score).create_dict(linkage_matrix,unique_ids)

In [ ]:
Evaluating_PM(score).cluster_stats(flat_pocket_paths_resid,cluster_dict)